In [1]:
Pipeline_Id = 1            
Pipeline_Name = "LoadingDataFromSourceToSilver" 


StatementMeta(sparkpool, 1, 2, Finished, Available, Finished)

In [2]:
jdbc_url = "jdbc:sqlserver://sqlserver8341.database.windows.net:1433;database=GoldLayer"
connection_properties = {
    "user": "suneetha",
    "password": "Suni@123",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

StatementMeta(sparkpool, 0, 3, Finished, Available, Finished)

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace,col,substring, substring_index, instr,concat_ws,md5,to_timestamp,lit,monotonically_increasing_id, lit,current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType,DateType,TimestampType
import re

source_schema = StructType([
                        StructField("id", IntegerType(), True),
                        StructField("batter", StringType(), True),
                        StructField("bowler", StringType(), True),
                        StructField("non_striker", StringType(), True),
                        StructField("runs_batter", IntegerType(), True),
                        StructField("runs_extras", IntegerType(), True),
                        StructField("runs_total", IntegerType(), True),
                        StructField("delivery_number", IntegerType(), True),
                        StructField("over_number", IntegerType(), True),  
                        StructField("team", StringType(), True),
                        StructField("inning_number", IntegerType(), True),
                        StructField("match_name", StringType(), True),
                        StructField("match_id", StringType(), True),
                        StructField("match_city", StringType(), True),
                        StructField("match_venue", StringType(), True),
                        StructField("match_type", StringType(), True),
                        StructField("team_type", StringType(), True),
                        StructField("match_start_date", DateType(), True),
                        StructField("player_out", StringType(), True),
                        StructField("_corrupt_record", StringType(), True)

                    ])


In [87]:
def process_files():
        mssparkutils.fs.mount(
            "abfss://bronzelayer@gen2storageforazurepoc.dfs.core.windows.net/",
            "/mountname",
            {"linkedService" : "synapsepoc123-WorkspaceDefaultStorage"}
        )

        files = mssparkutils.fs.ls(f'file:{mssparkutils.fs.getMountPath("/mountname")}')

        pattern = r"(\d{4}-\d{2}-\d{2}) (\d{2}:\d{2}:\d{2})"

        destination = spark.sql("SELECT COALESCE(MAX(date_time), CAST('1999-01-01' AS TIMESTAMP)) AS max_date_time FROM Cricket_Analysis_Data").collect()
        max_date_time = destination[0]["max_date_time"] if destination and destination[0]["max_date_time"] else None

        # looping each file in the directory 
        for file_info in files:
            file_path = file_info.path
            file_name = file_info.name      
            match = re.search(pattern, file_name)
            
            if match:
                date_time = f"{match.group(1)} {match.group(2)}"
            
                df = spark.createDataFrame([(file_name, date_time)], ["file_name", "date_time"])
                df = df.withColumn("date_time", to_timestamp("date_time", "yyyy-MM-dd HH:mm:ss"))

                if max_date_time:
                    if df.collect()[0]["date_time"] > max_date_time:
                        print(f"Source date_time ({df.collect()[0]['date_time']}) is newer. Proceed with processing {file_name}.")

                        path = f"abfss://bronzelayer@gen2storageforazurepoc.dfs.core.windows.net/{file_name}"
                        print(path)
                    
                        stage_df = spark.read.option("header","true").schema(source_schema).option("mode", "PERMISSIVE").option("columnNameOfCorruptRecord", "_corrupt_record").csv(path)
                        #display(stage_df)
                        
                        stage_df.cache()

                        cn_df = stage_df.where(col("_corrupt_record").isNotNull()).count()

                        if cn_df>0:

                            #Handling Corrupted Records...
                            print("Corrupted Records Found. Started loading into corruptedrecords container and also in process_corrupt_record table... ")

                            currupted_records= stage_df.where(col("_corrupt_record").isNotNull())
                            currupted_records.write.option("header",True).mode("append").csv("abfss://corruptedrecords@gen2storageforazurepoc.dfs.core.windows.net/corrupted records/")

                            currupted_records_df = stage_df \
                                .withColumn("Pipeline_Id", lit(Pipeline_Id)) \
                                .withColumn("Pipeline_Name", lit(Pipeline_Name)) \
                                .withColumn("file_name", lit(file_name)) \
                                .withColumn("path", lit(path)) \
                                .withColumn("date_time", to_timestamp(current_timestamp(), "yyyy-MM-dd HH:mm:ss")) \
                                .select("Pipeline_Id", "Pipeline_Name", "file_name", "path", "_corrupt_record", "date_time").where(col("_corrupt_record").isNotNull())

                            currupted_records_df.write.jdbc(url=jdbc_url, table="logs.process_corrupt_record", mode="append", properties=connection_properties)
                            print("Corrupted records loaded into a storage and processed_corrupt_record table successfully")
                        else:
                            print("No corrupted Records found")

                        # Processing Non corrupted records
                        print("Processing Non corrupted records.......")
                        source_df = stage_df.where(col("_corrupt_record").isNull()).drop("_corrupt_record")
                        display(source_df)

                        #extracting the date time from the file
                        file_path = f"{path}"
                        pattern = r"(\d{4}-\d{2}-\d{2}) (\d{2}:\d{2}:\d{2})"
                        match = re.search(pattern, file_path)

                        if match:
                            date_time = f"{match.group(1)} {match.group(2)}"
                            print(date_time)
                        
                        else:
                            print("No date and time found in the file name.")


                        # working on transformations
                        transforming_source_df = source_df.withColumn("batter", regexp_replace(col("batter"), "[^a-zA-Z0-9- ]", ""))\
                                            .withColumn("bowler", regexp_replace(col("bowler"), "[^a-zA-Z0-9- ]", ""))\
                                            .withColumn("non_striker", regexp_replace(col("non_striker"), "[^a-zA-Z0-9- ]", ""))\
                                            .withColumn("player_out", regexp_replace(col("player_out"), "-", ""))\
                                            .fillna({"runs_batter": 0, "runs_extras": 0,"runs_total": 0})\
                                            .withColumn("match_id", substring_index(col("match_id"), ".", 1))\
                                            .withColumn("date_time", to_timestamp(lit(date_time), "yyyy-MM-dd HH:mm:ss"))

                        #Generating hash key
                        df_with_hash = transforming_source_df.withColumn(
                            "hash_key", 
                            md5(
                                concat_ws(
                                    "_", 
                                    col("batter"),
                                    col("bowler"),
                                    col("non_striker"),
                                    col("runs_batter"),
                                    col("runs_extras"),
                                    col("runs_total"),
                                    col("delivery_number"),
                                    col("over_number"),
                                    col("inning_number"),
                                    col("match_name"),
                                    col("match_id"),
                                    col("match_city"),
                                    col("match_venue"),
                                    col("match_type"),
                                    col("team_type"),
                                    col("match_start_date"),
                                    col("player_out")
                                )
                            )
                        )
                        
                        #display(df_with_hash)
                        print("Hash code Generated")

                        df_with_hash.createOrReplaceGlobalTempView("df_with_hash_table")
                        print("Hash dataframe converted to temp table")

                        spark.sql(f"""MERGE INTO {dest_table_name} AS target
                        USING global_temp.df_with_hash_table AS source
                        ON target.hash_key = source.hash_key
                        WHEN NOT MATCHED THEN
                            INSERT (id, batter, bowler, non_striker, runs_batter, runs_extras, runs_total, delivery_number, over_number, team, inning_number, match_name, match_id, match_city, match_venue, match_type, team_type, match_start_date, player_out,date_time,hash_key)
                            VALUES (source.id, source.batter, source.bowler, source.non_striker, source.runs_batter, source.runs_extras, source.runs_total, source.delivery_number, source.over_number, source.team, source.inning_number, source.match_name, source.match_id, source.match_city, source.match_venue, source.match_type, source.team_type, source.match_start_date, source.player_out,source.date_time,source.hash_key);
                        """)
                        print(f"Inserted data into {dest_table_name}")
                         
                    else:
                        print("No need to process. Already processed Data")
                else:
                    print("No need to process")
            else:
                print(f"No date and time found in the file name: {file_name}")
            

In [89]:
dest_database_name = "default"
dest_schema_name = "dbo"
dest_table_name = "Cricket_Analysis_Data"

result = spark.sql(f"SHOW TABLES IN {dest_database_name}").filter(f"tableName = '{dest_table_name}'").count()

if result > 0:
    process_files()
else:
    schema = StructType([
                            StructField("id", IntegerType(), True),
                            StructField("batter", StringType(), True),
                            StructField("bowler", StringType(), True),
                            StructField("non_striker", StringType(), True),
                            StructField("runs_batter", IntegerType(), True),
                            StructField("runs_extras", IntegerType(), True),
                            StructField("runs_total", IntegerType(), True),
                            StructField("delivery_number", IntegerType(), True),
                            StructField("over_number", IntegerType(), True),  
                            StructField("team", StringType(), True),
                            StructField("inning_number", IntegerType(), True),
                            StructField("match_name", StringType(), True),
                            StructField("match_id", StringType(), True),
                            StructField("match_city", StringType(), True),
                            StructField("match_venue", StringType(), True),
                            StructField("match_type", StringType(), True),
                            StructField("team_type", StringType(), True),
                            StructField("match_start_date", DateType(), True),
                            StructField("player_out", StringType(), True),
                            StructField("date_time", TimestampType(), True),
                            StructField("hash_key", StringType(), True),
                        ])

    lake_df = spark.createDataFrame([], schema)
    lake_df.write.format("delta").mode("append").saveAsTable(f"{dest_table_name}")
    process_files()


In [2]:
jdbc_url = "jdbc:sqlserver://sqlserver8341.database.windows.net:1433;database=GoldLayer"
connection_properties = {
    "user": "suneetha",
    "password": "Suni@123",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

StatementMeta(sparkpool, 1, 3, Finished, Available, Finished)

In [14]:
Gold_Team_Read = spark.read.jdbc(url=jdbc_url, table="gold.gl_Team",  properties=connection_properties)
Gold_Team_Read.show()

<b>Team info</b>

In [3]:
from pyspark.sql import functions as F


# Load the data
df = spark.sql("""
    SELECT Team_Name, Team_Type 
        FROM (
        SELECT DISTINCT team AS Team_Name, team_type AS Team_Type, COUNT(*) AS con
        FROM cricket_analysis_data
        GROUP BY team, team_type
        ) AS teams_dataset
""")

Team_df_with_id = df.withColumn("Team_ID", F.abs(F.hash(F.col("Team_Name"), F.col("Team_Type"))))

Team_df = Team_df_with_id.select("Team_ID", "Team_Name", "Team_Type").distinct()

Gold_Team_Read = spark.read.jdbc(url=jdbc_url, table="[gold].[Team_Dim]",  properties=connection_properties)
Gold_Team_Read

Gold_Team_Join_df = Team_df.join(Gold_Team_Read, Team_df["Team_ID"] == Gold_Team_Read["Team_ID"],"left_anti")
Gold_Team_Join_df.show()

Gold_Team_Join_df.write.jdbc(url=jdbc_url, table="[gold].[Team_Dim]", mode="append", properties=connection_properties)
print("Loaded successfully")


StatementMeta(sparkpool, 1, 4, Finished, Available, Finished)

+-------+---------+---------+
|Team_ID|Team_Name|Team_Type|
+-------+---------+---------+
+-------+---------+---------+

Loaded successfully


<b>player info</b>

In [4]:
from pyspark.sql import functions as F

df = spark.sql("""
     SELECT DISTINCT player_name
        FROM (
            SELECT DISTINCT batter AS player_name FROM cricket_analysis_data
            UNION
            SELECT DISTINCT bowler AS player_name FROM cricket_analysis_data
        ) AS player_data
""")

player_df_with_id = df.withColumn("Player_ID", F.abs(F.hash(F.col("player_name"))))

players_df = player_df_with_id.select("Player_ID", "Player_Name").distinct()

#players_df.show()

Player_Gold_Read = spark.read.jdbc(url=jdbc_url, table="[gold].[Player_Dim]",  properties=connection_properties)
#Player_Gold_Read.show()

Player_Gold_Join_df = players_df.join(Player_Gold_Read, players_df["Player_ID"] == Player_Gold_Read["Player_ID"],"left_anti")
Player_Gold_Join_df.show()

Player_Gold_Join_df.write.jdbc(url=jdbc_url, table="[gold].[Player_Dim]", mode="append", properties=connection_properties)
print("Loaded successfully")


StatementMeta(sparkpool, 1, 5, Finished, Available, Finished)

+---------+-----------+
|Player_ID|Player_Name|
+---------+-----------+
+---------+-----------+

Loaded successfully


<b> relational table for players and team</b>

In [5]:
from pyspark.sql import functions as F

df = spark.sql("""
SELECT data.match_venue AS Match_Venue
	,data.match_city AS Match_City
FROM (
	SELECT DISTINCT match_venue
		,match_city
		,row_number() OVER (
			PARTITION BY match_venue ORDER BY match_city DESC
			) AS rownum
	FROM stage_match_details
    Where Match_Venue IS NOT NULL
	) AS data
WHERE rownum = 1
""")

Location_df_with_id = df.withColumn("Location_ID", F.abs(F.hash(F.col("match_venue"))))

Location_df = Location_df_with_id.select("Location_ID", "Match_Venue","Match_City").distinct()


# Reading data from the gold layer from location table

Location_Gold_Read = spark.read.jdbc(url=jdbc_url, table="[gold].[Location_Dim]",  properties=connection_properties)
Location_Gold_Read.show()


Location_Gold_Join_df = Location_df.join(Location_Gold_Read, Location_df["Location_ID"] == Location_Gold_Read["Location_ID"],"left_anti")
Location_Gold_Join_df.show()

Location_Gold_Join_df.write.jdbc(url=jdbc_url, table="[gold].[Location_Dim]", mode="append", properties=connection_properties)
print("Loaded successfully")




StatementMeta(sparkpool, 1, 6, Finished, Available, Finished)

+-----------+--------------------+--------------+
|Location_id|         Match_Venue|    Match_City|
+-----------+--------------------+--------------+
|   16343596|   Marsa Sports Club|         Marsa|
|   16899714|    St George's Park|Port Elizabeth|
|   28488780|         Manuka Oval|      Canberra|
|   47638255|        Ashlyns Road|Frinton-on-Sea|
|   54692628|Zahur Ahmed Chowd...|    Chittagong|
|   60546042|       Nehru Stadium|         Kochi|
|   61686815|County Ground, Derby|         Derby|
|   63718776|Rangiri Dambulla ...|          null|
|   66836724|Willowmoore Park ...|        Benoni|
|   85835543|King George V Spo...|        Castel|
|   93736385|St Lawrence Groun...|    Canterbury|
|   97112906|Maple Leaf North-...|     King City|
|  127138329|Saurashtra Cricke...|        Rajkot|
|  156973994|Pallekele Interna...|          null|
|  166891998|Subrata Roy Sahar...|          Pune|
|  167640842|         Seddon Park|      Hamilton|
|  182699840|Warner Park, Bass...|      St Kitts|


In [57]:
from pyspark.sql import functions as F

df = spark.sql("""
SELECT ca.match_id as Match_Id
	,ca.match_name as Match_Name
	,match_type as Match_Type
	,ca.Match_venue as Match_Venue
	,inning_number as Innings_Number
	,match_start_date as Match_Start_Date
FROM cricket_analysis_data ca
JOIN stage_Match_Location b ON ca.match_venue = b.match_venue

""")

Join_match_location = df.join(Location_df, df.Match_Venue == Location_df.Match_Venue,"inner")


Match_result = Join_match_location.select(
    df["Match_Id"],
    df["Match_Name"],
    df["Match_Type"],
	df["Innings_Number"],
	Location_df["Location_ID"],
    df["Match_Start_Date"]
).distinct()


Match_Gold_Read = spark.read.jdbc(url=jdbc_url, table="[Gold].[Match_Fact]",  properties=connection_properties)
Match_Gold_Read.show()

Match_Gold_Join_df = Match_result.join(Match_Gold_Read, Match_result["Match_Id"] == Match_Gold_Read["Match_Id"],"left_anti")
Match_Gold_Join_df.show()

Match_Gold_Join_df.write.jdbc(url=jdbc_url, table="[Gold].[Match_Fact]", mode="append", properties=connection_properties)

print("Loaded successfully")


StatementMeta(sparkpool, 1, 58, Finished, Available, Finished)

+--------+----------+----------+--------------+-----------+----------------+
|Match_Id|Match_Name|Match_Type|Innings_Number|Location_id|Match_Start_Date|
+--------+----------+----------+--------------+-----------+----------------+
+--------+----------+----------+--------------+-----------+----------------+

+--------+--------------------+----------+--------------+-----------+----------------+
|Match_Id|          Match_Name|Match_Type|Innings_Number|Location_ID|Match_Start_Date|
+--------+--------------------+----------+--------------+-----------+----------------+
| 1166914|Specsavers County...|       MDM|             2| 1099665471|      2019-04-05|
| 1094702|Caribbean Premier...|       T20|             1|  649402276|      2017-09-05|
|  576416|ICC World Cricket...|       ODM|             1| 2042149949|      2012-09-09|
|  693029|   NatWest T20 Blast|       T20|             2| 2140043680|      2014-05-18|
|  249749|ICC Champions Trophy|       ODI|             1| 1768679875|      2006-10

<b> Match Info</b>

In [23]:
from pyspark.sql import functions as F

df = spark.sql("""
     select DISTINCT player_name,team  FROM (
    select distinct batter as player_name,team from cricket_analysis_data
    UNION
    select distinct bowler as player_name,team from cricket_analysis_data
    ) as player_data 
""")


player_join_df = df.join(Team_df, df.team == Team_df.Team_Name,"inner") \
                 .join(players_df, df.player_name == players_df.Player_Name,"inner")


player_team_df = player_join_df.select("Player_ID", "Team_ID").distinct()

TeamPlay_Gold_Read = spark.read.jdbc(url=jdbc_url, table="[Gold].[Player_Team_Relational_dim]",  properties=connection_properties)

#TeamMatch_Gold_Read.show()


TeamPlay_Gold_Join_df = player_team_df.join(TeamPlay_Gold_Read, (player_team_df["Team_ID"] == TeamPlay_Gold_Read["Team_ID"]) & (player_team_df["Player_ID"] == TeamPlay_Gold_Read["Player_ID"]),"left_anti")

TeamPlay_Gold_Join_df.show()

TeamPlay_Gold_Join_df.write.jdbc(url=jdbc_url, table="[Gold].[Player_Team_Relational_dim]", mode="append", properties=connection_properties)

print("Loaded successfully")


StatementMeta(sparkpool, 1, 24, Finished, Available, Finished)

+----------+----------+
| Player_ID|   Team_ID|
+----------+----------+
| 577697921|1535997519|
|1628105426|1216931668|
| 860565847|1029878132|
|2066164182| 424095793|
| 931680392| 961597003|
| 862028888|1504815257|
|1385849740| 993809512|
|1841348667|1860782890|
| 362721381|1296592871|
|1649245999| 318216826|
|1498266766|1918973500|
|1261873785| 424095793|
| 950899226|1830915450|
|1611647657| 498005623|
|  10940854|1698303436|
|1508379357|1161712431|
|1890435699|1995353329|
|1233518762|1201572248|
| 474328113| 593509167|
|2013569934| 724494422|
+----------+----------+
only showing top 20 rows

Loaded successfully


In [61]:
from pyspark.sql import functions as F

df = spark.sql("""
SELECT match_id,Inning_Number as Innings_Number,team FROM cricket_analysis_data 
""")


Match_team_join_df = df.join(Team_df, df.team == Team_df.Team_Name,"inner")
#Match_team_join_df.show()

Match_Team_result = Match_team_join_df.select(
    df["Match_Id"],
    df["Innings_Number"],
    Team_df["Team_ID"]
)

#Match_Team_result.show()

TeamMatch_Gold_Read = spark.read.jdbc(url=jdbc_url, table="[Gold].[Team_Match_Relational_dim]",  properties=connection_properties)

#TeamMatch_Gold_Read.show()


TeamMatch_Gold_Join_df = Match_Team_result.join(TeamMatch_Gold_Read, (Match_Team_result["Team_ID"] == TeamMatch_Gold_Read["Team_ID"]) & (Match_Team_result["Match_ID"] == TeamMatch_Gold_Read["Match_ID"]),"left_anti")

TeamMatch_Gold_Join_df.show()

TeamMatch_Gold_Join_df.write.jdbc(url=jdbc_url, table="[Gold].[Team_Match_Relational_dim]", mode="append", properties=connection_properties)

print("Loaded successfully")

StatementMeta(sparkpool, 1, 62, Finished, Available, Finished)

+--------+--------------+---------+
|Match_Id|Innings_Number|  Team_ID|
+--------+--------------+---------+
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
| 1023591|             1|821019452|
+--------+--------------+---------+
only showing top 20 rows

Loaded successfully


<b> Creating Fact Table </b>

In [64]:
batter_fact = spark.sql('''
WITH aggregated_data AS (
    SELECT 
        batter,
        Match_Id,
		Inning_Number,
        SUM(Runs_Batter) AS sum_batter_runs,
        SUM(Runs_Total) AS total_score,
        COUNT(CASE WHEN runs_batter = 4 THEN 1 END) AS count_of_fours,
        COUNT(CASE WHEN runs_batter = 6 THEN 1 END) AS count_of_sixs,
		COUNT(CASE WHEN player_out is not null THEN 1 END) AS player_out
	    ,case when sum(Runs_Total) = 0 then 1 else 0 end duck_out

    FROM 
        cricket_analysis_data
    GROUP BY 
        batter, Match_Id,Inning_Number
)
SELECT 
    batter,
    sum_batter_runs,
    total_score,
    COALESCE(count_of_fours, 0) AS count_of_fours,
    COALESCE(count_of_sixs, 0) AS count_of_sixs,
	COALESCE(player_out, 0) AS player_out,
	duck_out
	,Match_Id
	,Inning_Number as Innings_Number
FROM aggregated_data

''')

#batter_fact.show()

batter_fact_join = batter_fact.join(players_df, players_df.Player_Name == batter_fact.batter,"inner")
#batter_fact_join.show()

rr = batter_fact_join.withColumnRenamed("player_ID","Batter_ID")

result = rr.select("Batter_ID","Match_Id","Innings_Number","sum_batter_runs","total_score","count_of_fours","count_of_sixs","player_out","duck_out").distinct()
result.show()

batterfact_Gold_Read = spark.read.jdbc(url=jdbc_url, table="[Gold].[Batter_Fact]",  properties=connection_properties)
#batterfact_Gold_Read.show()

batter_fact_Gold_Join_df = result.join(batterfact_Gold_Read, (result["Batter_ID"] == batterfact_Gold_Read["Batter_ID"]) & (result["Match_ID"] == batterfact_Gold_Read["Match_ID"]),"left_anti")

batter_fact_Gold_Join_df.show()

batter_fact_Gold_Join_df.write.jdbc(url=jdbc_url, table="[Gold].[Batter_Fact]", mode="append", properties=connection_properties)

print("Loaded successfully")

StatementMeta(sparkpool, 1, 65, Finished, Available, Finished)

+----------+--------+--------------+---------------+-----------+--------------+-------------+----------+--------+
| Batter_ID|Match_Id|Innings_Number|sum_batter_runs|total_score|count_of_fours|count_of_sixs|player_out|duck_out|
+----------+--------+--------------+---------------+-----------+--------------+-------------+----------+--------+
|1253226073|  946825|             2|             26|         26|             2|            0|         1|       0|
| 771742250|  531987|             3|            195|        205|            12|            5|         1|       0|
|1602648736|  804557|             1|             20|         22|             1|            1|         1|       0|
| 720368411| 1068642|             2|             17|         19|             2|            0|         1|       0|
|1261873785|  225254|             2|             23|         25|             4|            0|         0|       0|
|1347876109|  355996|             2|             20|         22|             1|         

In [69]:
bowler_df = spark.sql('''
WITH aggregated_data AS (
    SELECT 
        bowler,
        Match_Id,
		Inning_Number,
        SUM(Runs_Extras) AS run_extras,
        COUNT(CASE WHEN Player_Out is not null THEN 1 END) AS wicket
    FROM 
        cricket_analysis_data
    GROUP BY 
        bowler, Match_Id,Inning_Number
)
SELECT 
    bowler,
    Match_Id,
    Inning_Number as Innings_Number,
    run_extras as sum_of_run_extras,
    COALESCE(wicket, 0) AS wicket
FROM aggregated_data

''')


bowler_fact_join = bowler_df.join(players_df, players_df.Player_Name == bowler_df.bowler,"inner")
#batter_fact_join.show()

rr = bowler_fact_join.withColumnRenamed("player_ID","Bowler_ID")

result = rr.select("Bowler_ID","Match_Id","Innings_Number","sum_of_run_extras","wicket").distinct()
result.show()

Bowlerfact_Gold_Read = spark.read.jdbc(url=jdbc_url, table="[Gold].[Bowler_Fact]",  properties=connection_properties)
#batterfact_Gold_Read.show()

bowler_fact_Gold_Join_df = result.join(Bowlerfact_Gold_Read, (result["Bowler_ID"] == Bowlerfact_Gold_Read["Bowler_ID"]) & (result["Match_ID"] == Bowlerfact_Gold_Read["Match_ID"]),"left_anti")

bowler_fact_Gold_Join_df.show()

bowler_fact_Gold_Join_df.write.jdbc(url=jdbc_url, table="[Gold].[Bowler_Fact]", mode="append", properties=connection_properties)

print("Loaded successfully")


StatementMeta(sparkpool, 1, 70, Finished, Available, Finished)

+----------+--------+--------------+-----------------+------+
| Bowler_ID|Match_Id|Innings_Number|sum_of_run_extras|wicket|
+----------+--------+--------------+-----------------+------+
| 392674578| 1167146|             1|                0|     3|
|1127955263|  455237|             2|                3|     4|
|1667068185|  267710|             1|                2|     2|
| 510427160|  433570|             1|                0|     1|
|1820583710| 1114865|             1|                0|     1|
|1448751226|  875477|             2|                1|     4|
|1879849745|  692759|             1|                0|     0|
| 128119192| 1197142|             1|                2|     0|
| 890680799|  478279|             1|                1|     3|
|2015376879|  576416|             1|               12|     4|
|1361316829| 1254058|             1|                0|     0|
|1859946190|  531629|             2|                4|     3|
|1381923431| 1166965|             2|                6|     1|
| 425461